<a href="https://colab.research.google.com/github/tejasp2022/IntroNeuralNetworks/blob/master/PS7_(CLEAN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Set 7: Backpropagation
# CMSC 422, Fall 2020
# Due Nov 20 at 11:59pm

<center>
<img src="https://miro.medium.com/max/1914/1*F9capAHwl_rz2-Q8z511WQ.jpeg" alt="meme" width="500px"/>
</center>

# Instructions
In this problem set you will implement backpropagation for a set of different neural network architectures.
There is some code provided for you here, and you will write your implementations in the places marked with __```#TODO: Your Code Here```__. You may add helper functions if you feel you need to.

__Analysis Questions:__ In addition to Python programming, each problem will contain some analysis questions (under __Analysis__). These are meant to ensure you understand your results, and will be manually graded on Gradescope.

__Submission:__ download this notebook as a `.ipynb` file and submit it to Gradescope. This assignment will be partially autograded so follow instructions closely.  
 
- Make sure your plots are visible when downloading the notebook, otherwise they won't appear on Gradescope. 
- Make sure your code cells are not throwing exceptions.
- Please do not import any packages other than what has already been imported here. You may be penalized for doing so.
- Lastly, the autograder times out after 40 minutes, so make sure your implementation is relativly efficient (e.g. by using numpy for matrix operations). Our implementation took a little over 10 minutes to test.

# Problems

## Problem 1 (25 Points)
We'll begin with the simplest possible network (a single layer perceptron). It has a single input feature that we call $x$. This is the activation of the single node of the input layer. This is connected to a single output node, which has a weight, $w$. We also have a bias term, so the activation of the output unit is $a = wx + b$. This network will be used to solve a linear regression problem. So, if we are given an input pair of $(x,y)$, we want to minimize the squared loss: 

$$L(x,y) = (a-y)^2 = (wx + b - y)^2$$

To do this, you will need to randomly initialize the weight and bias and then perform gradient descent.
<br>
<br>
<center>
<img src="https://drive.google.com/thumbnail?id=1Qz8jJaPXbVzoL44Nd4nbQgFHOA8G_XyI&sz=w1000" alt="net1" width="150px"/>
<br>
<i>Figure 1: Network for Problem 1</i>
</center>
<br>
<br>
The gradient of the loss is computed using a training set containing pairs, $(x_1, y_1), (x_2, y_2), ... (x_n, y_n)$. We have:

$$\nabla L = \frac{1}{n} \sum_{i=1}^n \left( \frac{\partial L}{\partial w}(x_i, y_i), \frac{\partial L}{\partial b}(x_i, y_i)  \right)$$

If we denote $\theta = (w,b)$ as a vector containing all the parameters of the network, we perform gradient
descent with the update:

$$\theta^k = \theta^{k-1} - \eta \nabla L$$

Here $\eta$ is the learning rate, and $\theta^k$ denotes a vector of $(w,b)$ after the $k$'th iteration of gradient descent. Do not mistake $\eta$ (the learning rate) for $n$ (the number of data points).
We provide you with a routine to generate training data. This has the form: 

```simplest_training_data(n)```  
  
This just generates $n$ random training points on the line $y = 3x + 2$, with a little Gaussian noise added to the points.  
You need to write a routine with the form: 

```simplest_training(n, k, eta)```

Here $n$ indicates the number of points in the training set (you can call `simplest_training_data` to get the training data), $k$ indicates the total number of iterations that you will use in training, and $\eta$ is the learning rate.  To initialize the weights in your network, we suggest that you initialize $w$ with a Gaussian random variable with mean 0 and variance of 1, and that you initialize $b = 0$.  
You also need to write a routine of the form: 

```simplest_testing(theta, x)```

This routine applies the network, using the parameters in theta, to the input values in the vector $x$, and returns a vector of results in $y$.
After training, the network should learn $w$ and $b$ values that are similar to those used to train the network.  So you can test your network by looking at the learned $w$ and $b$ values.  Or you can use the testing algorithm to see if the network computes appropriate $y$ values.  In testing, you may find that if you use too big a value for $\eta$ the network will not converge to anything meaningful.  If you use a value of $k$ that is too small, it won't have time to converge to a good solution.
We run our algorithm with $n = 30, k = 10,000, \eta = .02$.   When we test using $x = (0, 1, ..., 9)$ we get the result:
  
```
1.9504  4.9666  7.9828  10.9990  14.0152  17.0314  20.0476  23.0638  26.0800  29.0962
```

These points fit the line $y = 3x + 2$.

In [ ]:
import numpy as np
import math as m
import sys

###Problem 1
###Provided function to create training data
def simplest_training_data(n):
    m = 3
    b = 2
    x = np.random.uniform(0,1,n)
    y = m*x+b+0.3*np.random.normal(0,1,n)
    return (x,y)

def simplest_training(n, k, eta):
  return theta 


def simplest_testing(theta, x):
  # TODO: implement this method
  return y

### Analysis (10 Points)
Test your algorithm using the test mentioned above or any other test you choose. Provide a description of your test (including the values chosen/used) and your results inside this cell.  

- !!! _YOUR RESPONSE HERE_ !!!

## Problem 2 (35 Points)
You will now create a network that is a little more complicated. It still contains just an input and an output layer, with no hidden layers. But it now has a nonlinearity along with a cross-entropy loss, so that we can use it for classification.
<br>
<br>
<center>
<img src="https://drive.google.com/thumbnail?id=1UkNx6-HghYRsjrXbqB6jN04VUsA-_RKp&sz=w1000" alt="net2" width="400px"/>
<br>
<i>Figure 2: Network for Problem 2</i>
</center>
<br>

The network has two inputs, $x_1$ and $x_2$.  These are connected with two weights to a single output unit.  If we let $z = w_1x_1 + w_2x_2 + b$, the output unit will have an activation of $a = \sigma(z)$, where $\sigma(z)$ represents the sigmoid function:

$$\sigma(z) = \frac{1}{1+e^{-z}}$$

We can interpret the output as giving the probability that the input belongs to class 1. If the probability is low, then the input probably belongs to class 0. Hint: the derivative of the sigmoid is given by:

$$\frac{d\sigma}{dz} = \sigma(z)(1-\sigma(z))$$

In training the network, you will use the cross-entropy loss. In this case, the cross entropy loss will be:

$$L_{CE}(x,y) = -(y\log{a} + (1-y)\log{(1-a)})$$

If $y = 1$, this is just the negative log of what the network predicts for the probability that the input belongs to class 1.  If $y = 0$, it is the negative log of the probability that the input belongs to class 0.
We provide you with a routine to generate training data. This has the form:

```single_layer_training_data(trainset)```  
    
which returns $X$ and $y$.
This provides two different training sets.  When the input, trainset, is 1, the function produces a simple, linearly separable training set.  Half the points are near $(0,0)$ and half are near $(10,10)$.  $X$ is a matrix in which each row contains one of these points, so it is $n \times 2$, where $n$ is the number of points.  $y$ is a vector of class labels, which have the value 1 for the points near $(0,0)$ and 0 for the points near $(10,10)$.

When trainset is 2, we generate a different training set that is not linearly separable, but that corresponds to the Xor problem.  Points from class 1 are either near $(0,0)$ or $(10,10)$, while points in class 0 are near either $(10,0)$ or $(0,10)$.

You will need to implement two routines.  
The first is: 

```single_layer_training(k, eta, trainset)```  
  
As before, $k$ will indicate the number of iterations of gradient descent and eta gives the learning rate.  trainset indicates which training set to use, 1 or 2.  You will train the network using the same gradient descent approach as in the previous problem.  As before, we suggest that you initialize weights using random values chosen from a Gaussian distribution with zero mean, and that you initialize bias at 0.  

You will also implement a test routine: 

```single_layer_testing(theta, X)```  
  
This takes in the network parameters and a matrix, $X$, of the form returned by single\_layer\_training\_data.  It returns a vector of the output values the network computes.

__Remember__: The `trainset` argument is the integer to be used to generate data with `single_layer_training_data(trainset)`, it is NOT the training dataset.

In [ ]:
###Problem 2
###Provided function to create training data
def single_layer_training_data(trainset):
    n = 10
    if trainset == 1:
    # Linearly separable
        X = np.concatenate((np.random.normal((0,0),1,(n,2)), np.random.normal((10,10),1,(n,2))),axis=0)
        y = np.concatenate((np.ones(n), np.zeros(n)),axis=0)

    elif trainset == 2:
        # Not Linearly Separable
        X = np.concatenate((np.random.normal((0,0),1,(n,2)), np.random.normal((10,10),1,(n,2)), np.random.normal((10,0),1,(n,2)), np.random.normal((0,10),1,(n,2))),axis=0)
        y = np.concatenate((np.ones(2*n), np.zeros(2*n)), axis=0)

    else:
        print ("function single_layer_training_data undefined for input", trainset)
        sys.exit()

    return (X,y)

def single_layer_training(k, eta, trainset):
  #TODO: Your Code Here
  return theta

def single_layer_testing(theta, X):
  #TODO: Your Code Here
  return y

### Analysis (10 Points)
Add a description (inside this cell) of tests of your code using the two trainset values.  You will need to figure out how many iterations of gradient descent to perform and the appropriate learning rate to get good results (mention these values and include the learned weights).  Do not use the data you used to train the network, call `single_layer_training_data` again to get fresh data for testing.  When trainset = 1, your network should assign a high probability of belonging to class 1 for points near $(0,0)$, and a low probability for points near $(10,10)$.  When trainset = 2, the data is not linearly separable, so you may find that your network has problems being able to separate.  Describe what happens in both cases.

- !!! _YOUR RESPONSE HERE_ !!!

## Problem 3 (40 Points)
<center>
<img src="https://drive.google.com/thumbnail?id=1lZQ1CnQUDD-kiyL-FtDu0UTO1dmEy-Oq&sz=w1000" alt="net3" width="400px"/>
<br>
<i>Figure 3: Network for Problem 3</i>
</center>
<br>
<br>
Now you will implement a multi-layer network that has a hidden layer.  To start with a relatively simple case, we will do this without any non-linearities. The network has two input units, $x_1$ and $x_2$.  These are connected to a single hidden unit.  We'll call the activation of this hidden unit $h$, so $h = w_{11}x_1 + w_{12}x_2 + b_{11}$.  This hidden unit is connected to two output units.  We'll call their activation $z_1$ and $z_2$, so we have:

$$z_1 = w_{21}h + b_{21}~~~~~~~~~~~z_2 = w_{22}h + b_{22}$$

To train this network, we use a loss function that says that we want the output to be close to the input.  So the loss function is:  
  
$$L(x_1, x_2) = (z_1 - x_1)^2 + (z_2 - x_2)^2$$
    
That is, the input is also acting as the label.  This kind of network is called an {\em auto-encoder}.  You may be wondering what the point of this is.  Because the hidden layer is smaller than the input and output layers, the network is forced to learn low-dimensional representation of the data.  In this case, the network learns to map the input points onto a line in the hidden layer, and then compute the 2D coordinates of the points on this line for the output layer.  This process is called Principal Component Analysis (PCA), and we will learn more about it later in the semester.

We will provide a routine to generate training data:

```pca_training_data(n, sigma)```  
  
The input parameter $n$ indicates the number of points in the training set.  As in the last problem, $X$ contains a $n \times 2$ matrix in which each row contains the coordinates of a 2D point.  These points are generated to lie along the line $y = x + 1$.  Then Gaussian noise is added to the points, with zero mean and a standard deviation of sigma.

Once again, you will implement training and testing routines.  
  
`pca_training(k, eta, n, sigma)`  
  
The input $k$ gives the number of iterations of gradient descent to use, while $eta$ gives the learning rate.  The input value $n$ indicates the number of points in the training set, while $sigma$ indicates the amount of noise added to these points.  Use these as parameters to pca\_training\_data.  The routine returns theta, a representation of all the weights and biases in the network.
Also implement a test routine: 

```pca_test(theta, X)```  
  
$X$ will contain test data in the form returned by pca\_training\_data.  $Z$ provides the results the network produces given this input; $Z$ has the same format as $X$.  

To test this, try training the network with $n = 10$ and $sigma = .1$.  Then test, using the input: `pca_test(theta, [[1,2], [4,5], [10, 3]])`.  
  
When I run my  code with this test I get: `[[0.9418, 2.0653], [3.9543, 5.0511], [6.1780, 7.2551]]`.  



In [ ]:
###Problem 3
###Provided function to create training data
def pca_training_data(n, sigma):
    m = 1
    b = 1
    x1 = np.random.uniform(0,10,n)
    x2 = m*x1+b
    X = np.c_[x1, x2]
    X += np.random.normal(0, sigma, X.shape)
    return X

def pca_training(k, eta, n, sigma):
    #TODO: Your Code Here
    return theta

def pca_test(theta, X):
    #TODO: Your Code Here
    return Z

### Analysis (10 Points)
It may take a little work to find good values for $k$ and $\eta$.  Add a description of your experimental results inside this cell.  Can you explain why the network produces the point $(6.1780, 7.2551)$ with an input of $(10, 3)$?

Do another test with $sigma = 0$ instead of $sigma = .1$.  Run your network with the same test data.  How have the results changed?  Can you explain this change?

- !!! _YOUR RESPONSE HERE_ !!!

## Problem 4 (optional challenge problem, for extra credit, 20 points):
Ok, now you are ready to create a complete, fully connected neural net with a hidden layer and non-linearities. You will use this network to solve the XOR problem, using the same training data as in Problem 2. Your network architecture should have the following components:
- Two input units, with activations $x_1$ and $x_2$.  These are just the coordinates of 2D points.
- A variable number of hidden units, H.  Write your code so that you can select the number of hidden units as a hyperparameter.  Let's call the activation of the $i$'th hidden unit, $a^1_i$.  Let's call the weights of these units $w^1_{ij}$.  This is the weight from input unit $j$ to hidden unit $i$.  
- se a RELU non-linearity for the hidden units.  So to determine the activation of a hidden unit we have: $z^1_i = w^1_{i1}x_1 + w^1_{i2}x_2 + b^1_i$, and $a^1_i = max(0, z^1_i)$.
- There is then a single output unit.  Call its activation $a^2$.  We compute this as: $z^2 = \left( \sum_{i=1}^H w^2_{1i}a^1_i \right) + b^2$, and $a^2 = \sigma(z^2)$, where $\sigma$ is the sigmoid nonlinearity.  This last part is just the same as in Problem 2.  And, like Problem 2, you can train your network using the cross-entropy loss.

<center>
<img src="https://drive.google.com/thumbnail?id=1qfLXZJsDFIwTfIdHxEP6HnJBmUrPGZsT&sz=w1000" alt="net3" width="400px"/>
<br>
<i>Figure 4: Network for Problem 4</i>
</center>
<br>
<br>

Implement test and training functions with the templates:

`nn_training(k, eta, trainset, H)`

`nn_testing(theta, X)`

The parameters to the training routine are similar to those in Problem 2, with $H$ indicating the number of hidden units.  The testing routine has the same form as in Problem 2.

__Remember__: The `trainset` argument is the integer to be used to generate data with `single_layer_training_data(trainset)`, it is not the actual training dataset.

In [ ]:
###Problem 4: Challenge Problem
def nn_training(k, eta, trainset, H):
    #TODO: Your Code Here
    return theta

def nn_testing(theta, X):
    #TODO: Your Code Here
    return y

### Analysis (5 Points)
Run experiments to demonstrate that your network can solve the XOR problem. How do you find the results vary as you vary the number of hidden units?  Show and discuss the results of your experiments inside/below this cell.

- !!! _YOUR RESPONSE HERE_ !!!